In [177]:
from project import *
import csv
import mysql.connector as sql
from mysql.connector import Error
import re

In [171]:
clear_database(cursor, dbConn, tables)

In [ ]:
#get connection 
dbConn = create_connection("localhost", "root", "123456", "progDB")
cursor = dbConn.cursor()
create_database(cursor, "progDB")
create_tables_from_file(cursor, "test_schema.sql", dbConn)

In [168]:
# create the tables from file

def create_tables_from_file(cursor, filename, connection):
    '''Note: if schema does not inlcude "IF NOT EXISTS" error occurs
    if the table already exists in the database
    '''
    try:
        with open(filename,'r') as file:
            sql_script = file.read()
            
        for result in cursor.execute(sql_script, multi=True):
            pass
    except Error as e:
        print(f"The error '{e} occurred'")
    
    connection.commit()
    

## General Database Management

In [162]:
tables = ["Program", "Department", "Faculty", "Course", "Section", 
          "LearningObjective", "SubObjective", "CourseEval",
          "SectionEval", "ProgramObjective"]

def clear_database(cursor, connection, tableList):
    try:
        cursor.execute("SET FOREIGN_KEY_CHECKS = 0")
        for table in tableList:
            sqlcom = "DROP TABLE IF EXISTS %s" % (table)
            cursor.execute(sqlcom)
        cursor.execute("SET FOREIGN_KEY_CHECKS = 1")
    except Error as e:
        print(f"The error '{e} occurred'")
    connection.commit()

In [176]:
def check_dept_exists(dept_code):
    try:
        com = "SELECT * FROM Department WHERE DepartmentCode = '%s'" % (dept_code) 
        cursor.execute(com)
        
        if cursor.fetchall():
            return True
        
        print("Department does not exist. Please check department code.")
        return False
    except Error as e:
        print(f"Error executing Department Select: {e}")
        
        
def check_fal_exists(fal_id):
    try:
        com = "SELECT * FROM Faculty WHERE FacultyID = '%s'" % (fal_id) 
        cursor.execute(com)
        
        if cursor.fetchall():
            return True
        
        print("Faculty member does not exist. Please check faculty id")
        return False
    except Error as e:
        print(f"Error executing Faculty Select: {e}")
        return False
        
def check_program_exists(prog_name):
    try:
        com = "SELECT * FROM Program WHERE ProgramName = '%s'" % (prog_name) 
        cursor.execute(com)
        
        if cursor.fetchall():
            return True
        
        print("Program does not exist. Please check program name")
        return False
    except Error as e:
        print(f"Error executing Program Select: {e}")
        return False
    
def enter_program_info(cursor, connection, prog_name, pc_id, dept_code):
    '''Check to see if program coordinator exists, if not add them to the faculty table'''
    
    #check if department exists, if not, return error 
    if not check_dept_exists(dept_code):
        return False
    
    if not check_fal_exists(pc_id):
        return False
    
    try:
        # Insert data into the Programs table
        sqlcom = "INSERT INTO Program \
        (ProgramName, ProgramCoordinatorID, DepartmentCode) \
        VALUES (%s, %s, %s)"
        
        program_data = (prog_name, pc_id, dept_code)
        
        cursor.execute(sqlcom, program_data)
        # Commit the transaction
        connection.commit()

        print(f"Data for program '{prog_name}' successfully inserted into the Programs table.")
        return True
    except Error as e:
        print(f"Error inserting program data: {e}")
        return False
    

def enter_department_info(cursor, connection, dept_code, dept_name):
    try:
        # Insert data into the Departments table
        sqlcom = "INSERT INTO Department (DepartmentCode, DepartmentName) \
            VALUES (%s, %s)"
        dept_info = (dept_code, dept_name)
        cursor.execute(sqlcom, dept_info)

        # Commit the transaction
        connection.commit()

        print(f"Data for department '{dept_name}' successfully inserted into the Departments table.")
        return True
    except Error as e:
        print(f"Error inserting department data: {e}")
        return False

def enter_faculty_info(cursor, connection, fac_id, fac_name, fac_email, dept_code, fac_rank):    
    try:
        if not check_dept_exists(dept_code):
            return False
    
        # Insert data into the Faculty table
        sqlcom = "INSERT INTO Faculty (FacultyID, FacultyName, FacultyEmail, FacultyRank, DepartmentCode) \
        VALUES (%s, %s, %s, %s, %s)"
        faculty_data = (fac_id, fac_name, fac_email, fac_rank, dept_code)
        cursor.execute(sqlcom, faculty_data)

        # Commit the transaction
        connection.commit()

        print(f"Data for faculty '{fac_name}' successfully inserted into the Faculty table.")
        return True
    except Error as e:
        print(f"Error inserting faculty data: {e}")
        return False

def enter_course_info(cursor, connection, co_id, co_title, co_desc, dept_code):
    if not check_dept_exists(dept_code):
        return False
    
    try:
        Insert data into the Courses table
        sqlcom = "INSERT INTO Course (CourseID, CourseTitle, CourseDescription, DepartmentCode) \
            VALUES (%s, %s, %s, %s)"
        course_data = (co_id, co_title, co_desc, dept_code)
        cursor.execute(sqlcom, course_data)

        # Commit the transaction
        connection.commit()

        print(f"Data for course '{co_title}' successfully inserted into the Courses table.")
        return True
    except Error as e:
        print(f"Error inserting course data: {e}")
        return False
    
'''Assign Course to Program'''
def assign_course_to_program(cursor, connector, course_id, prog_name):
    # check if course and program exists
    if not check_course_exists(course_id):
        return False
    if not check_program_exists(prog_name):
        return False

    try:
        #insert data into CourseEval table
        cursor.execute("INSERT INTO ProgramCourse (CourseID, ProgramName) VALUES (?,?)",
                       course_id, prog_name)
        # commit changes to the database 
        connector.commit()
        return True
    except Error as e:
        print(f"Error inserting Learning Objectives data: {e}")
        return False
    
'''Assigning learning (sub)objectives to (course, program) pairs (remember, a course can
be associated with multiple programs, and for each program, the objectives can be
different).'''
def assign_obj_to_course(cursor, connection, co_id, prog_name, obj_code):
    # check if course, program pair exists
    com = "SELECT * FROM ProgramCourse WHERE ProgramName = '%s' AND CourseID = '%s'" % (prog_name, co_id)
    cursor.execute(com)
    
    if cursor.fetchall(): 
        try:
            sqlcom = "INSERT INTO ProgramCourse (CourseID, ProgramName) \
                VALUES (%s, %s)"
            course_data = (co_id, prog_name)
            cursor.execute(sqlcom, course_data)

            # Commit the transaction
            connection.commit()
            return True
        except Error as e:
            print(f"Error inserting Program Course data: {e}")
            return False
    return False

In [173]:
enter_department_info(cursor, dbConn, "0000", "Test Department")
enter_faculty_info(cursor, dbConn, "1111111", "Test Falc", "faltest@gmail.com", "0000", "Adjunct")
enter_program_info(cursor, dbConn, "Test 2 Program", "1111111", "0000")
enter_faculty_info(cursor, dbConn, "1111113", "Test Falc 3", "faltest3@gmail.com", "0000", "boop")
enter_course_info(cursor, dbConn, "CS1111", "Test Course", "hello this course is for dunces like you", "0000")

Data for department 'Test Department' successfully inserted into the Departments table.
Data for faculty 'Test Falc' successfully inserted into the Faculty table.
Data for program 'Test 2 Program' successfully inserted into the Programs table.
Error inserting faculty data: 3819 (HY000): Check constraint 'faculty_chk_1' is violated.
Data for course 'Test Course' successfully inserted into the Courses table.


True

In [160]:
# cursor.execute("SELECT COUNT(*) FROM Faculty WHERE FacultyId = '1111111'")

# #print(cursor.fetchall())

# num = cursor.fetchall()
# print(num)

# print(type(num[0][0]))

[(1,)]
<class 'int'>


## User Input Handling
- check emails
- delete all extra spaces 

In [152]:
def verify_email(email):
    email_pattern = re.compile(r'^[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}$')
    match = re.match(email_pattern, email)
    return bool(match)

def check_dept_code(dept_code):
    dept_pattern = re.compile(r'^[a-zA-Z]{4}$')
    match = re.match(dept_pattern, dept_code)
    return bool(match)

def check_course_id(c_id):
    id_pattern = re.compile(r'^[a-zA-Z]{4}[0-9]{4}$')
    match = re.match(id_pattern, c_id)
    return bool(match)

In [164]:
def handle_program_entry(cursor, connection, name, pc_id, dept_code):
    if not check_dept_code(dept_code):
        return "Error: Department Code not valid"
    
    if enter_program_info(cursor, connection, str(name), str(pc_id), str(dept_code)):
        return "Program Info Stored Successfully"
    else:
        return "There was an issue entering the information, please try again."
    
def handle_department_entry(cursor, connection, dept_code, dept_name):
    if not check_dept_code(dept_code):
        return "Error: Department Code not valid"
    
    if enter_department_info(cursor, connection, str(dept_code), str(dept_name)):
        return "Department Info Stored Successfully"
    else:
        return "There was an issue entering the information, please try again."
    
def handle_faculty_entry(cursor, connection, fac_id, fac_name, fac_email, dept_code, fac_rank = "Adjunct"):
    if not check_dept_code(dept_code):
        return "Departement Code not valid"
    
    if verify_email(fac_email):
        if enter_faculty_info(cursor, connection, fac_id, fac_name, fac_email, dept_code, fac_rank):
            return "Faculty Info Stored Successfully"
    else:
        return "Email is not valid. Please try again."

    
def handle_course_entry(cursor, connection, co_id, co_title, co_desc, dept_code):
    if not check_dept_code(dept_code) and check_course_id(co_id):
        return "Department Code or Course ID not valid"
    
    if enter_course_info(cursor, connection, str(co_id), str(co_title), str(co_desc), str(dept_code)):
        return "Course Info Stored Successfully"